In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [5]:
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [8]:
## Save the encoders and scaler
with open('label_encoder_gender_reg.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo_reg.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler_reg.pkl','wb') as file:
    pickle.dump(scaler,file)

In [9]:
#ANN Regression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [11]:
#Build the model
model=Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  #output layer
])

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="regression_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [13]:
history=model.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step - loss: 100358.0156 - mae: 100358.0156 - val_loss: 98447.0703 - val_mae: 98447.0703
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 99386.6953 - mae: 99386.6953 - val_loss: 96519.8125 - val_mae: 96519.8125
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - loss: 96041.6875 - mae: 96041.6875 - val_loss: 91690.7344 - val_mae: 91690.7344
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 89640.7812 - mae: 89640.7812 - val_loss: 83927.3516 - val_mae: 83927.3516
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - loss: 80695.8906 - mae: 80695.8906 - val_loss: 74455.3203 - val_mae: 74455.3203
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - loss: 70876.3281 - mae: 70876.3281 - val_loss: 65392.2734 - val_mae: 65392.2734
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - loss: 62141.8828 - mae: 62141.8828 - val_loss: 58264.2617 - val_mae: 58264.2617
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━

In [14]:
model.save('model_reg.h5')
#model.save("model_tf", save_format="tf")

In [15]:
%load_ext tensorboard

In [17]:
%tensorboard --logdir regression_logs/fit

Reusing TensorBoard on port 6006 (pid 4745), started 0:00:02 ago. (Use '!kill 4745' to kill it.)

In [48]:
#model evaluation
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - loss: 50256.5391 - mae: 50256.5391
Test MAE: 50256.5390625
